In [123]:
%matplotlib inline

In [124]:
import numpy as np
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nilearn.input_data import NiftiMasker
from scipy.stats import stats

In [125]:
def sigmoid(energy):
    X = (energy - np.nanmean(energy)) / np.nanstd(energy)
    sig = 1 / (1 + np.exp(-X))
    return sig

In [126]:
df = pd.read_csv("/home/julia/data/gradients/genes/gene_list_full.csv")
df = df[df['acronym']!="FALSE"]
df = df[np.isnan(df['entrez'])==False]
df = df.reset_index(drop=True)

In [127]:
df_energy = pd.read_csv('/home/julia/data/gradients/genes/energy.csv', index_col=0)

In [128]:
mod = []
for a in df['acronym']:
    if ('Ht' in a):
        mod.append(a)
    elif  ('Drd' in a):
        mod.append(a)
    elif ('Chrn' in a) or ('Chrm' in a):
        mod.append(a)
    elif ('Adra' in a) or ('Adrb' in a):
        mod.append(a)

In [155]:
mask_img = nb.load('/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight_200um.nii.gz')
mask = mask_img.get_data()
aff = mask_img.affine
hdr = mask_img.header

In [130]:
cortex = np.where(mask==1)[0].shape[0]

In [131]:
masker = NiftiMasker(mask_img=mask_img, standardize=False, smoothing_fwhm=0.45)

In [132]:
energy = np.zeros((cortex, len(mod)))

for a in range(len(mod)): 
    idcs = df.loc[df['acronym']==mod[a], 'id'].values
    
    for i in idcs:
        gene = np.zeros(mask.shape)
        gene += nb.load('/home/julia/data/gradients/genes/energy/%s/energy.nii.gz'%i).get_data()
        #gene = sigmoid(gene)

    gene = gene / len(idcs) 
    energy[:,a] = gene[mask==1]
    energy[:,a][energy[:,a]==-1]=np.nan
    #masker.fit_transform(nb.Nifti1Image(gene, aff, hdr))

In [152]:
pcas = nb.load('/home/julia/data/gradients/results/genes/pca_img.nii.gz').get_data()

In [165]:
for e in range(energy.shape[1]):
    for p in range(2):
        pca_masked = pcas[:,:,:,p][mask==1]
        e_data = np.where(np.isnan(energy[:,e])==False)
        r = stats.pearsonr(pca_masked[e_data], energy[:,e][e_data])
        if r[0] > 0.2:
            print(e,p,r)

10 1 (0.28349330777928067, 4.2276825507387616e-228)
12 1 (0.21927810222381783, 4.6879598727496785e-149)
14 1 (0.2580408749075597, 4.698889727782286e-233)
21 1 (0.44301897117840067, 0.0)
22 0 (0.22387253701292423, 3.881933666092581e-174)
24 1 (0.3720455525286698, 0.0)
25 1 (0.2940891316002039, 4.375972125169218e-305)
29 1 (0.33547802544027105, 0.0)
30 1 (0.3085611684929892, 0.0)
31 0 (0.2812665220248364, 7.393557441401504e-270)
31 1 (0.2182271236931492, 1.2682578084923352e-160)
